In [ ]:
import numpy as np
import pickle
import cv2
from os import listdir
from sklearn.preprocessing import LabelBinarizer
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation, Flatten, Dropout, Dense
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.preprocessing import image
from keras.preprocessing.image import img_to_array
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import pandas as pd


Using TensorFlow backend.


In [ ]:
import tensorflow as ts
import keras as ks

In [ ]:
print(ts.__version__)

1.14.0


In [ ]:
default_image_size = tuple((256, 256))
## Function to convert image to array of 256 X 256
def convert_image_to_array(image_dir):
    try:
        image = cv2.imread(image_dir)
        if image is not None :
            image = cv2.resize(image, default_image_size)   
            return img_to_array(image)
        else :
            return np.array([])
    except Exception as e:
        print(f"Error : {e}")
        return None

In [ ]:
image_list_a, label_list = [], []

totalImage = 0  # it is used to control total numbe of images for each class

root_dir = listdir("Downloads\Plant_Disease/") # Specify location of plant images

for image_folder in root_dir :  # Look for each folder
    # Look for All images in each folder
    image_list = listdir(f"Downloads\Plant_Disease/{image_folder}")
    
    for image in image_list:
        # Read each image in a variable image_list_a and 
        #correspoding folder name as image lable
        
        image_name = f"Downloads\Plant_Disease/{image_folder}/"+image
        if isinstance(image_name, str) == True:
            if image_name.endswith(".jpg") == True or image_name.endswith(".JPG") == True:
                image_list_a.append(convert_image_to_array(image_name))
                label_list.append(image_folder)
        totalImage = totalImage+1
        if(totalImage>=20):          
            break
            
print(len(label_list),len(np.unique(np.asarray(label_list))))
from sys import getsizeof
getsizeof(image_list_a)

33 15


344

In [ ]:
label_binarizer = LabelBinarizer()
image_labels = label_binarizer.fit_transform(label_list)
pickle.dump(label_binarizer,open('label_transform.pkl', 'wb'))
n_classes = len(label_binarizer.classes_)

In [ ]:
# Split data in train and test
image_list_a = np.asarray(image_list_a)
x_train, x_test, y_train, y_test = train_test_split(image_list_a, image_labels, test_size=0.2, random_state = 42)

In [ ]:
EPOCHS = 10
INIT_LR = 1e-3
BS = 32
    width=256
    height=256
    depth=3
    inputShape = (height, width, depth)
    input_shape = (x_train.shape[1], x_train.shape[2], x_train.shape[3]) 
chanDim = 1

# Develope  model
model = Sequential()
model.add(Conv2D(4, (3, 3), input_shape=inputShape))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=1))
model.add(MaxPooling2D(pool_size=(3, 3)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(524))
model.add(Activation("relu"))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(n_classes))
model.add(Activation("softmax"))
model.summary()


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 254, 254, 4)       112       
_________________________________________________________________
activation_1 (Activation)    (None, 254, 254, 4)       0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 254, 254, 4)       1016      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 84, 84, 4)         0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 84, 84, 4)         0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 28224)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 524)             

In [ ]:
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(loss="binary_crossentropy", optimizer=opt,metrics=["accuracy"])

In [ ]:
history = model.fit(x_train, y_train, batch_size=32,
    validation_data=(x_test, y_test),
    epochs=EPOCHS, verbose=1
    )

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Train on 26 samples, validate on 7 samples
Epoch 1/10
26/26 [==============================] - 3s 111ms/step - loss: 0.3122 - accuracy: 0.9077 - val_loss: 2.0505 - val_accuracy: 0.8667
Epoch 2/10
26/26 [==============================] - 1s 53ms/step - loss: 0.0966 - accuracy: 0.9692 - val_loss: 1.9966 - val_accuracy: 0.8667
Epoch 3/10
26/26 [==============================] - 1s 55ms/step - loss: 0.0336 - accuracy: 0.9897 - val_loss: 2.0505 - val_accuracy: 0.8667
Epoch 4/10
26/26 [==============================] - 2s 58ms/step - loss: 0.0146 - accuracy: 0.9974 - val_loss: 2.0505 - val_accuracy: 0.8667
Epoch 5/10
26/26 [==============================] - 1s 54ms/step - loss: 0.0060 - accuracy: 1.0000 - val_loss: 2.0505 - val_accuracy: 0.8667
Epoch 6/10
26/26 [==============================] - 1s 57ms/step - loss: 0.0049 - accuracy: 1.0000 - val_loss: 2.0505 - val_accuracy: 0.8667
Epoch 7/10
26/2

In [ ]:
model.evaluate(x_test, y_test)

7/7 [==============================] - 0s 17ms/step


[1.961829423904419, 0.8666667342185974]